In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
print(nb_dir)
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
# autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
autolib_dir = '/'.join(nb_dir.split('/')[:-1])
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload
%autoreload 2

/home/kai/data/shiyi/AlphaBoosting/automl/automl_app
['', '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload', '/home/kai/.local/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions', '/home/kai/.ipython', '/home/kai/data/shiyi/AlphaBoosting/automl']


In [2]:
from automl_libs import base_layer_utils, stacknet
from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels, nn_libs 

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']

In [21]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list, low, high):
    features_to_gen = []
    for function in function_list:
        for i in range(low, high+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, 
#                                        fe.cumulative_count, 
#                                        fe.reverse_cumulative_count, 
#                                        fe.variance, fe.count_std_over_mean], 
#                                       2, 2)# len(categorical_features))
# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count])
features_to_gen = get_features_to_gen([fe.count, fe.unique_count], 2, 3)#len(categorical_features))

features_to_gen

[{'feature_cols': ['Month', 'DayofMonth'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DepTime'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'UniqueCarrier'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'Origin'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'Dest'],
  'function': <function automl_libs.feature_engineering.co

In [22]:
len(features_to_gen)

168

In [6]:
project_path = './' 

In [7]:
from automl_app import logger_config
import logging
logger_config.config(project_path+'log_project2.log', file_loglevel=logging.INFO)

In [8]:
import numpy as np
print(np.random.rand())

0.0008781695396854472


In [14]:
def gs_params_gen(model='lgb'):
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    if model == 'svc':
        params = {
            'C': 0.911#np.random.rand(),
#             'metric': 'roc_auc'
        }
    elif model == 'logreg':
        params = {
            'penalty': np.random.choice(['l2']),
            'dual': np.random.choice([True, False]),
            'C': 1,#np.random.rand(),
            'metric': 'roc_auc'
        }
    elif model == 'lgb':
        params = {    
            'objective': 'binary',
            'boosting': 'gbdt',
            'num_boost_round': 30, # ignored in params. extract it and put it in input arguments in train or cv explicitly
                                   # seems to work fine as the upper limit when combined with early_stopping_round
            'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
            'num_leaves': np.random.choice([15,31]),#,61,127]),
            'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
            'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
            'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
            'feature_fraction': np.random.randint(9,11)/10,
            'feature_fraction_seed': seed,
            'early_stopping_round':10,
            'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
            'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
            'bagging_seed': seed,
            'scale_pos_weight': 1,
            'metric': 'auc' 
        }
    elif model == 'nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': 1,
            'lr_init': 0.01,
            'lr_fin': 0.005, # if == lr_init, then no lr decay
            'batch_size': np.random.choice([32, 64, 128, 256, 512, 1024, 2048]),
            "pred_batch_size": 50000,
            'max_ep': 1,
            'patience': 1,
            'cat_emb_outdim': 50, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
            'dense_units': 50,
            'num_dense_n_layers': 1,
            'drop_rate': 0.2,
            'combined_dense_n_layers': 1,
            'monitor': 'val_auc',  # or val_loss (MUST HAVE)
            'mode': 'max'  # MUST HAVE
        }
    elif model == 'nn_layer2':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': 1,
            'lr_init': 0.01,
            'lr_fin': 0.01, # if == lr_init, then no lr decay
            'batch_size': 128,
            "pred_batch_size": 50000,
            'max_ep': 1,
            'patience': 1,
            'cat_emb_outdim': 50, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
            'dense_units': 50,
            'num_dense_n_layers': 1,
            'drop_rate': 0.2,
            'combined_dense_n_layers': 1,
            'monitor': 'val_auc',  # or val_loss (MUST HAVE)
            'mode': 'max',  # MUST HAVE
            'categorical_feature': []
        }
    return params, seed

In [10]:
gs_params_gen('nn_layer2')

({'batch_size': 128,
  'cat_emb_outdim': 50,
  'categorical_feature': [],
  'combined_dense_n_layers': 1,
  'dense_units': 50,
  'drop_rate': 0.2,
  'ep_for_lr': 1,
  'lr_fin': 0.01,
  'lr_init': 0.01,
  'max_ep': 1,
  'mode': 'max',
  'monitor': 'val_auc',
  'nn_seed': 14167431,
  'num_dense_n_layers': 1,
  'patience': 1,
  'pred_batch_size': 50000},
 14167422)

In [23]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'last_run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, gs_params_gen)

2018-07-06 14:19:38 | INFO    | automl_app.app.AlphaBoosting.__init__             | #22  | ==========NEW RUN==========
2018-07-06 14:19:38 | INFO    | automl_app.app.AlphaBoosting.__init__             | #65  | Run record file [./output/last_run_record.json] not found. Begin the first time run...
2018-07-06 14:19:38 | INFO    | automl_app.app.AlphaBoosting.__init__             | #96  | generate todo list
2018-07-06 14:19:38 | INFO    | automl_app.app.AlphaBoosting.__init__             | #112 | STAGE: FEATURE_ENGINEERING
2018-07-06 14:19:40 | DEBUG   | automl_libs.feature_engineering.count                | #65  | feature generated: count__Month_DayofMonth
2018-07-06 14:19:41 | DEBUG   | automl_libs.feature_engineering.count                | #65  | feature generated: count__Month_DayOfWeek
2018-07-06 14:19:43 | DEBUG   | automl_libs.feature_engineering.count                | #65  | feature generated: count__Month_DepTime
2018-07-06 14:19:43 | DEBUG   | automl_libs.feature_engineering.coun

2018-07-06 14:20:22 | DEBUG   | automl_libs.feature_engineering.count                | #65  | feature generated: count__DayofMonth_DepTime_Origin
2018-07-06 14:20:24 | DEBUG   | automl_libs.feature_engineering.count                | #65  | feature generated: count__DayofMonth_DepTime_Dest
2018-07-06 14:20:25 | DEBUG   | automl_libs.feature_engineering.count                | #65  | feature generated: count__DayofMonth_DepTime_Distance
2018-07-06 14:20:26 | DEBUG   | automl_libs.feature_engineering.count                | #65  | feature generated: count__DayofMonth_UniqueCarrier_Origin
2018-07-06 14:20:27 | DEBUG   | automl_libs.feature_engineering.count                | #65  | feature generated: count__DayofMonth_UniqueCarrier_Dest
2018-07-06 14:20:29 | DEBUG   | automl_libs.feature_engineering.count                | #65  | feature generated: count__DayofMonth_UniqueCarrier_Distance
2018-07-06 14:20:30 | DEBUG   | automl_libs.feature_engineering.count                | #65  | feature gene

2018-07-06 14:21:11 | DEBUG   | automl_libs.feature_engineering.unique_count         | #129 | feature generated: unique_count__Month_DayofMonth_DayOfWeek
2018-07-06 14:21:11 | DEBUG   | automl_libs.feature_engineering.unique_count         | #129 | feature generated: unique_count__Month_DayofMonth_DepTime
2018-07-06 14:21:12 | DEBUG   | automl_libs.feature_engineering.unique_count         | #129 | feature generated: unique_count__Month_DayofMonth_UniqueCarrier
2018-07-06 14:21:12 | DEBUG   | automl_libs.feature_engineering.unique_count         | #129 | feature generated: unique_count__Month_DayofMonth_Origin
2018-07-06 14:21:13 | DEBUG   | automl_libs.feature_engineering.unique_count         | #129 | feature generated: unique_count__Month_DayofMonth_Dest
2018-07-06 14:21:13 | DEBUG   | automl_libs.feature_engineering.unique_count         | #129 | feature generated: unique_count__Month_DayofMonth_Distance
2018-07-06 14:21:14 | DEBUG   | automl_libs.feature_engineering.unique_count       

2018-07-06 14:21:39 | DEBUG   | automl_libs.feature_engineering.unique_count         | #129 | feature generated: unique_count__UniqueCarrier_Dest_Distance
2018-07-06 14:21:39 | DEBUG   | automl_libs.feature_engineering.unique_count         | #129 | feature generated: unique_count__Origin_Dest_Distance
2018-07-06 14:21:39 | INFO    | automl_app.app.AlphaBoosting.__init__             | #116 | STAGE: VALIDATION_DOWNSAMPLING_GEN
2018-07-06 14:22:24 | INFO    | automl_app.app.AlphaBoosting.__init__             | #120 | STAGE: CONCAT_DATA
2018-07-06 14:22:34 | INFO    | automl_app.app.AlphaBoosting.__init__             | #124 | STAGE: GRID_SEARCH


> /home/kai/data/shiyi/AlphaBoosting/automl/automl_app/app.py(323)_get_final_data()
-> feature_cols = list(set(train.columns) - set(not_features) - set(label_col_as_list))
(Pdb) train.shape
(950000, 177)
(Pdb) train.to_pickle('train_more177.pkl')
(Pdb) test.to_pickle('test_more177.pkl')
(Pdb) test.shape
(100000, 176)
(Pdb) q


BdbQuit: 